In [2]:
import pickle
import pandas as pd
from spellchecker import SpellChecker
from tqdm import tqdm
import re
import string
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0


import os
import stanza # Stanford's stanza package
#stanza.download('en') # run this once
import pandas as pd
import numpy as np
import textstat
import pickle
from tqdm import tqdm

In [3]:
allSessions = pickle.load( open( "../Data/DataSets/SWC/SWC.p", "rb" ) )
#allSessions = pickle.load( open( "../../thesis/Data/Session/allSessionsProc.p", "rb" ) )
allSessionsSQS = list(pickle.load( open( "../Data/DataSets/SQS/SQS.p", "rb" ) ))
allQueries = allSessions['query'].tolist() 
allQueries = allQueries + list(allSessionsSQS)
allQueries = set(allQueries)

In [7]:
print(len(allSessionsSQS))

1505


In [3]:
kidsMispelled = []
count = 0
import csv
with open('DataSets/KidSpell/Web_Search_Lab_Errors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if count == 0:
            count += 1
        else:
            kidsMispelled.append(row[0])

count = 0
with open('DataSets/KidSpell/Web_Search_Informal_Errors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:

        if count == 0:
            count += 1
        else:
            kidsMispelled.append(row[0])
    
count = 0
with open('DataSets/KidSpell/Essay_Writing_Errors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:

        if count == 0:
            count += 1
        else:
            kidsMispelled.append(row[1])

kidsMispelled = set(kidsMispelled)

In [4]:
kidsMispelled

{'',
 'sentenses',
 'sam',
 'tak',
 'en',
 'thir',
 'cip',
 'wus',
 'realy',
 'priins',
 'shi',
 'peise',
 'desine',
 'stared',
 "din't",
 'bother',
 'ticettes',
 'runing',
 'taset',
 'nois',
 'oll',
 'winn',
 'controled',
 'set',
 'frst',
 'riped',
 'anmls',
 'scientce',
 'stoveris',
 'explist',
 'seld',
 'crecher',
 'hoomadrobots',
 'costum',
 'wor',
 'gymnasteks',
 'mose',
 'summor',
 'vidio',
 'tings',
 'day',
 'tokes',
 'travling',
 'teri',
 'egsist',
 'mavi',
 'miyols',
 'eliffent',
 'perfermer',
 'chude',
 'swors',
 'samon',
 'neighboors',
 'donuts',
 'tonit',
 'whate',
 'labron',
 'numbr',
 'cum',
 'hase',
 'dolfen',
 'villajg',
 'yus',
 'utack',
 'scorpen',
 'cheses',
 'Texes',
 'sos',
 'stichs',
 'sqweeking',
 'perpol',
 'smel',
 'fot',
 'fand',
 'ice crem',
 'gos',
 'woulb',
 'peik',
 'cockotate',
 'blud',
 'tat',
 'shoo',
 'gunn',
 'gay',
 'exsmstopm',
 'favrot',
 'drot',
 'cin',
 'yowst',
 'onece',
 'wons',
 'to',
 'win',
 'wiht',
 'journel',
 'magick',
 'oferd',
 'tastes'

In [5]:
spell = SpellChecker()
spellingError = []
oneOffError = []
kidsError = []
spellingErrorAvg = []
oneOffErrorAvg = []
kidsErrorAvg = []
netModifiers = ["www", "com", "net", "gov", "edu", "org", 'http']
with tqdm(total = len(allQueries) ) as pbar:
    for query in allQueries:
        query =  query.translate(str.maketrans('', '', string.punctuation))
        res = [ele for ele in netModifiers if(ele in query)] 
        if not res:
            try:
                lang = detect(query)
                length = len(query.split(" "))
                misspelled = spell.unknown(query.split(" "))
                found = 0
                oneOff = 0
                kidsMis = 0
                for word in misspelled:
                    if word in kidsMispelled:
                        kidsMis +=1
                    candid=spell.candidates(word)
                    edits = spell.edit_distance_1(word)
                    for can in candid:
                        if can in edits:
                            oneOff += 1
                            break
                oneOffError.append(oneOff)
                spellingError.append(len(misspelled))
                kidsError.append(kidsMis)
                spellingErrorAvg.append(len(misspelled)/length)
                oneOffErrorAvg.append(oneOff/length)
                kidsErrorAvg.append(kidsMis/length)
            except:
                oneOffError.append(-1)
                spellingError.append(-1)
                kidsError.append(-1)
                spellingErrorAvg.append(-1)
                oneOffErrorAvg.append(-1)
                kidsErrorAvg.append(-1)

        else:
            spellingError.append(0)
            oneOffError.append(0)
            kidsError.append(0)
            spellingErrorAvg.append(0)
            oneOffErrorAvg.append(0)
            kidsErrorAvg.append(0)
        pbar.update()

100%|██████████| 70621/70621 [3:00:43<00:00,  6.51it/s]   


In [6]:
spelling = pd.DataFrame(data=spellingError, columns = ['numSpellingErrors'])
spelling['query'] = allQueries
spelling['offByOne'] = oneOffError
spelling['kidsError'] = kidsError
spelling = spelling.set_index('query')

In [7]:
invalidcharacters= set(['!', ',', '.', '?'])
webops = ['www', 'http', 'com', 'net','edu','org', 'gov' ,'co','mil']
punct = []
casing = []
for query in allQueries:

    if any(char in invalidcharacters for char in query):
        if any(substring in query for substring in webops):
            punct.append(0)
        else:
            punct.append(1)
    else:
        punct.append(0)
        
    if query.islower():
        casing.append(0)
    else:
        casing.append(1)

In [8]:
spelling['punct'] = punct
spelling['casing'] = casing

In [9]:
spelling = spelling.reset_index()

In [10]:
pickle.dump(spelling, open( "Pickles/SPFeat.p", "wb" ) )